## Select temperatures using linear regression

In [9]:
import requests as rq
import pandas as pd
import os
import warnings
from datetime import timedelta
from sklearn import preprocessing
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.stats import f
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor as RF

In [2]:
warnings.filterwarnings('ignore')

In [5]:
end = '20190918T23-05'
start = '20130918T23-05'
key = '8a6315646d5695061696c71a041c42c0'
series_id = 'EBA.NW-ALL.D.HL'
data = rq.get("http://api.eia.gov/series/?api_key={}&series_id={}&start={}&end={}".format(key, series_id, start, end))
hourly_demand = data.json()['series'][0]['data']
electricity_demand = pd.DataFrame(hourly_demand, columns=['datetime','usage'])
# electricity_demand = pd.read_csv('electricity_data/electricity_demand.csv', index_col=0)

In [6]:
electricity_demand['datetime'] = pd.to_datetime(electricity_demand['datetime'], format='%Y%m%dT%H-%M')
electricity_demand['time'] = electricity_demand['datetime'].apply(lambda x:x.hour)
electricity_demand['month'] = electricity_demand['datetime'].apply(lambda x:x.month)
electricity_demand['weekday'] = electricity_demand['datetime'].apply(lambda x:x.weekday())
electricity_demand['year'] = electricity_demand['datetime'].apply(lambda x:x.year)
electricity_demand['day'] = electricity_demand['datetime'].apply(lambda x:x.day)

# Normalize the usage
electricity_demand['normalized_usage'] = preprocessing.MinMaxScaler().fit_transform(electricity_demand['usage'].values.reshape(-1,1))

In [10]:
#weather_2013 = pd.read_csv('electricity_data/hourly_temperature_2013.csv')
#weather_2015 = pd.read_csv('electricity_data/hourly_temperature_2015.csv')
#weather_2016 = pd.read_csv('electricity_data/hourly_temperature_2016.csv')
#weather = pd.concat([weather_2013, weather_2015, weather_2016])

# Other weather data.
# temp_dir = 'electricity_data/temperatures/'
temp_dir = 'electricity_data/NW/'
# stations = [72243012960, 72259003927, 72253012921]
stations = []
cities = []
# weather = pd.DataFrame()

for file in os.listdir(temp_dir):
    if file.endswith('.csv') and file != "Bend.csv":
        cities.append(file.split('.')[0])
        #temp_df = pd.read_csv(temp_dir+file)
        #stations.append(temp_df.head(1)['STATION'].values[0])
        #weather = pd.concat([weather, temp_df], axis=0)

In [8]:
# Many columns has very few useful rows. Collect only useful columns.
columns = weather.columns

useful_columns = []
for column in columns:
    if sum(weather[column].isna()) > 30000:
        # There are more than 10000 nan
        print('{} is abondoned, containing {} NaN'.format(column, sum(weather[column].isna())))
    else:
        useful_columns.append(column)
        
weather = weather[useful_columns]

AWND is abondoned, containing 796753 NaN
BackupDirection is abondoned, containing 430135 NaN
BackupDistance is abondoned, containing 430135 NaN
BackupDistanceUnit is abondoned, containing 430135 NaN
BackupElements is abondoned, containing 371586 NaN
BackupElevation is abondoned, containing 371586 NaN
BackupElevationUnit is abondoned, containing 797402 NaN
BackupEquipment is abondoned, containing 430135 NaN
BackupLatitude is abondoned, containing 371586 NaN
BackupLongitude is abondoned, containing 371586 NaN
BackupName is abondoned, containing 371586 NaN
CDSD is abondoned, containing 796754 NaN
CLDD is abondoned, containing 796754 NaN
DSNW is abondoned, containing 796846 NaN
DailyAverageDewPointTemperature is abondoned, containing 777675 NaN
DailyAverageDryBulbTemperature is abondoned, containing 777364 NaN
DailyAverageRelativeHumidity is abondoned, containing 777649 NaN
DailyAverageSeaLevelPressure is abondoned, containing 777702 NaN
DailyAverageStationPressure is abondoned, containing

In [9]:
def round_time(date_time):
    minute = date_time.minute
    if minute > 30:
        return date_time+timedelta(hours=1)
    else:
        return date_time
    
def parse_temperature(s):
    try:
        return float(s)
    except:
        try:
            return float(re.split('[a-z]+', s)[0])
        except:
            return np.nan

In [10]:
# Round the date info
weather['datetime'] = pd.to_datetime(weather['DATE'], format='%Y-%m-%dT%H:%M:%S')
weather['date'] = weather['datetime'].apply(lambda date_time: round_time(date_time))
weather['time'] = weather['date'].apply(lambda x:x.hour)
weather['month'] = weather['date'].apply(lambda x:x.month)
weather['day'] = weather['date'].apply(lambda x:x.day)
# weather['weekday'] = weather['date'].apply(lambda x:x.weekday())
weather['year'] = weather['date'].apply(lambda x:x.year)
weather['HourlyDryBulbTemperature'] = weather['HourlyDryBulbTemperature'].apply(lambda s:parse_temperature(s))

In [11]:
# Get rid of nan and duplicated values
match_columns = ['year', 'month', 'day', 'time']
useful_columns = ['STATION', 'DATE', 'HourlyDryBulbTemperature', 'month', 'day', 'time', 'year']
weather = weather.loc[weather['HourlyDryBulbTemperature'].notna(),:]
weather = weather[useful_columns].groupby(match_columns+['STATION'], as_index=False).mean()

In [14]:
# bad_station = [72232403071]
bad_station = [72063800224]
df_joined = electricity_demand


for station in stations:
    if station not in bad_station:
        df_tmp = weather.loc[weather['STATION'] == station, :]
        if sum(df_tmp['HourlyDryBulbTemperature'].isna()) != 0:
            print(station)
        old_num_rows = len(df_joined)
        df_joined = df_joined.merge(df_tmp, how='inner', left_on=match_columns, right_on=match_columns)
        new_num_rows = len(df_joined)
        if new_num_rows < old_num_rows-100:
            print(station)

72569024089
72464093058


In [15]:
new_column_names = ['datetime', 'usage', 'time', 'month', 'weekday', 'year', 'day',
       'normalized_usage', 'STATION_1', 'T_1',
       'STATION_2', 'T_2', 'STATION_3', 'T_3', 'STATION_4', 'T_4',
       'STATION_5', 'T_5', 'STATION_6', 'T_6', 'STATION_7', 'T_7',
       'STATION_8', 'T_8', 'STATION_9', 'T_9', 'STATION_10', 'T_10',
       'STATION_11', 'T_11', 'STATION_12', 'T_12', 'STATION_13', 'T_13']
df_joined.columns = new_column_names

In [36]:
def feature_selection_using_LR(X, y):
    num_features = X.shape[1]
    features_ordered = []
    RSS_0 = np.sum((y-np.mean(y))**2)
    min_p = 0
    max_f = 1
    while max_f > 0.5: # The Null hypothesis is rejected.
        max_f = -np.inf
        min_p = np.inf # Find the next figure producing the maximum p value.
        next_feature = -1
        min_RSS = np.inf
        
        for ind in np.arange(num_features):
            if ind in features_ordered:
                continue
            y_pred = LinearRegression().fit(X[:,features_ordered+[ind]],y).predict(X[:,features_ordered+[ind]])
            RSS_1 = np.sum((y-y_pred)**2)
            
            degree = X.shape[0]-len(features_ordered)-1
            f_stat = (RSS_0-RSS_1)*degree/RSS_1 # The f statistics
            p_value = f.sf(f_stat, 1, degree)
            p_value = 2*(p_value if p_value<0.5 else 1-p_value)
            '''
            if p_value < min_p:
                min_p = p_value
                next_feature = ind
                min_RSS = RSS_1
            '''
                
            if f_stat > max_f:
                max_f = f_stat
                next_feature = ind
                min_RSS = RSS_1
                
        features_ordered.append(next_feature)
        # print('Feature {} is included with a P value {}.'.format(next_feature, min_p))
        print('Feature {} is included with a f statistics {}.'.format(next_feature, max_f))
        RSS_0 = min_RSS
    return features_ordered

In [37]:
temp_features = []

for i in np.arange(1,14):
    temp_features.append('T_'+str(i))
    
features_ordered = feature_selection_using_LR(df_joined[temp_features].values, df_joined['usage'].values)

Feature 1 is included with a f statistics 1831.0221428624195.
Feature 4 is included with a f statistics 2440.307131065897.
Feature 8 is included with a f statistics 336.51340248871685.
Feature 6 is included with a f statistics 243.19692118225814.
Feature 12 is included with a f statistics 126.83928628787747.
Feature 2 is included with a f statistics 114.96709620810812.
Feature 10 is included with a f statistics 124.66488764830426.
Feature 0 is included with a f statistics 124.36233087311439.
Feature 3 is included with a f statistics 168.2604706749979.
Feature 11 is included with a f statistics 28.70938451047781.
Feature 9 is included with a f statistics 15.583150482260166.
Feature 7 is included with a f statistics 0.5619519033995625.
Feature 5 is included with a f statistics 0.04115025845023452.


In [23]:
df_joined.columns

Index(['datetime', 'usage', 'time', 'month', 'weekday', 'year', 'day',
       'normalized_usage', 'STATION_1', 'T_1', 'STATION_2', 'T_2', 'STATION_3',
       'T_3', 'STATION_4', 'T_4', 'STATION_5', 'T_5', 'STATION_6', 'T_6',
       'STATION_7', 'T_7', 'STATION_8', 'T_8', 'STATION_9', 'T_9',
       'STATION_10', 'T_10', 'STATION_11', 'T_11', 'STATION_12', 'T_12',
       'STATION_13', 'T_13'],
      dtype='object')

In [38]:
for ind in features_ordered:
    print(cities[ind])

Redmond
Spokane
Eugene
Boise
Portland
Billings
Pueblo
Denver
Casper
Great_Falls
Seattle
Pocatello
Salt_Lake_City


In [124]:
# print the stations that are important
print(np.asarray(temp_features)[features_ordered[:-1]])

['T_1' 'T_2' 'T_5' 'T_12' 'T_4' 'T_7' 'T_6' 'T_9' 'T_8' 'T_3' 'T_11'
 'T_10']


In [126]:
np.asarray(stations)[features_ordered[:-1]]

array([72243012960, 72259003927, 72363023047, 72092600313, 72242012923,
       72250612959, 72259453909, 72254013904, 72096513910, 72253012921,
       72267023042, 72232403071])

'Houston', 'Dallas', 'Amarillo', 'Beaumont', 'Galveston', 'Macallen', 'Fort Worth', 'Austin', 'Abilene', 'San Antonio', 'Lubbock', 'Midland'

## Feature selection using random forest

In [39]:
def attenuate(X,Y,beta,column):
    # The first 24 hour will be used as warm up.
    X_new = (1-beta)*X[24:, column:].copy()
    # X_new += 0.9*X[23:-1,column:]+0.9*X[22:-2,column:]+0.9*X[21:-3,column:]+0.9*X[20:-4,column:]# +0.9*X[19:-5,column:]+0.9*X[18:-6,column:]
    # for h in np.arange(1, 6):
        # X_new = np.concatenate((X_new, X[24-h:-1*h,column:]), axis=1)
    for h in np.arange(1, 24):
        X_new += beta**h*(1-beta)*X[24-h:-1*h, column:]
    return np.concatenate((X[24:,:column], X_new), axis=1), Y[24:]

In [40]:
# Smooth the temperature.
beta = 0.84
column = 3
all_features = ['time','weekday','month']+temp_features
X_rf, y_rf = df_joined[all_features].values, df_joined['usage'].values
X_rf, y_rf = attenuate(X_rf, y_rf, beta, column)

num_samples = len(y_rf)
split_point = int(num_samples*2/3)
train_ind = np.arange(split_point)
valid_ind = np.arange(split_point, num_samples)

In [41]:
from sklearn import base
from sklearn.metrics import make_scorer

class elec_forcast(base.BaseEstimator):
    
    def __init__(self, n_estimators=700, min_samples_split=4, min_samples_leaf=2, random_state=10):
        self.n_estimators = n_estimators
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state
        self.rf = RF(n_estimators=self.n_estimators, min_samples_split=self.min_samples_split, \
                    min_samples_leaf=self.min_samples_leaf, \
                    random_state=self.random_state)
        
    def get_params(self, deep=True):
        return {"n_estimators": self.n_estimators,
                "min_samples_split":self.min_samples_split,
                "min_samples_leaf": self.min_samples_leaf,
                "random_state": self.random_state}
        
    def fit(self, X, y):
        self.rf.fit(X,y)
        
    def predict(self, X):
        return self.rf.predict(X)        
        
    def score(self, X, Y_true):
        Y_pred = self.predict(X)
        Y_pred = LinearRegression().fit(Y_pred.reshape(-1,1), Y_true.reshape(-1,1)).predict(Y_pred.reshape(-1,1))
        return r2_score(Y_true, Y_pred)

In [48]:
def feature_selection_using_rf(X_rf, y_rf, train_ind, vaild_ind):
    grid = {'n_estimators': [50],
       'min_samples_split': [4,6],
       'min_samples_leaf': [1,2],
       'random_state': [10]}
    
    num_features = X_rf.shape[1]
    features_ordered = [0, 8, 1, 4, 2, 3, 9, 5, 10] #[]
    r2_diff = 1
    old_r2 = 0
    while r2_diff > -1: # The Null hypothesis is rejected.
        # Find the next feature producing the r2 score.
        next_feature = -1
        max_r2 = -np.inf
        
        for ind in np.arange(num_features):
            if ind in features_ordered:
                continue
            # Generate a r2 score for the validation set after gridsearch.
            search = GridSearchCV(elec_forcast(), grid, cv=[(train_ind, valid_ind)], n_jobs=4)
            search.fit(X_rf[:,features_ordered+[ind]], y_rf)            
            
            r2 = search.best_score_
            
            if r2 > max_r2:
                max_r2 = r2
                next_feature = ind
                
        features_ordered.append(next_feature)
        print('Feature {} is included with a r2 score {}.'.format(next_feature, max_r2))
        r2_diff = max_r2 - old_r2
        old_r2 = max_r2
    return features_ordered

In [49]:
feature_selection_using_rf(X_rf, y_rf, train_ind, valid_ind)

Feature 6 is included with a r2 score 0.8355740679371391.
Feature 13 is included with a r2 score 0.834832680349859.
Feature 15 is included with a r2 score 0.8345312661411547.
Feature 12 is included with a r2 score 0.8337528874895903.
Feature 14 is included with a r2 score 0.833028331338397.
Feature 11 is included with a r2 score 0.8317148652439311.
Feature 7 is included with a r2 score 0.8289771327884842.
Feature -1 is included with a r2 score -inf.


[0, 8, 1, 4, 2, 3, 9, 5, 10, 6, 13, 15, 12, 14, 11, 7, -1]

In [3]:
features_ordered = [0, 8, 1, 4, 2, 3, 9, 5, 10, 6, 13, 15, 12, 14, 11, 7, -1]

In [11]:
for ind in features_ordered:
    if ind > 2:
        print(cities[ind-3])

Salt_Lake_City
Redmond
Denver
Boise
Billings
Pocatello
Casper
Pueblo
Portland
Seattle
Great_Falls
Eugene
Spokane


In [53]:
features_ordered

[1, 4, 8, 6, 12, 2, 10, 0, 3, 11, 9, 7, 5]